# 60 minutos sobre XGBoost

> My confession: I (over)use GBM. When in doubt, use GBM.

-- Owen Zhang, 2015;[ _"Winning Data Science Competitions"_ @ NYC Data Science Academy](https://www.youtube.com/watch?v=LgLcfZjNF44)


## Israel Zúñiga de la Mora | 01/Mar/202 @ Saturdays.ai Mty


# Objetivo de esta plática:
Conocer el uso básico de la biblioteca de Machine Lerning XGBoost para Gradient Boosting en Python.



# Que no veremos hoy (AKA: the proof is left as an exercise to the reader):
- Uso avanzado de XGBoost:
    - Selección de features por puntuación (XGBoost + Scikit-Learn's `SelectFromModel` class)
    - Monitoreo de performance en entrenamiento y _Early Stopping_
    - Tuning: Multithreading, Gradient Boosting Estimators, Learning Rate
    - Serialización de modelos (Pickle/Joblib)
- Uso y/o despliegue a Producción en AWS/GCP/Azure/Kubeflow/Polyaxon/MLFlow
- Comparación con otras implementaciones similares (Microsoft's LightGBM / Yandex's CatBoost)
- [Teoría Matemática/CompSci sobre algoritmos implementados](https://www.slideshare.net/ShangxuanZhang/kaggle-winning-solution-xgboost-algorithm-let-us-learn-from-its-author)
    
    
<img src="https://ae01.alicdn.com/kf/HTB1_u80dNWYBuNjy1zkq6xGGpXaF/Mini-pollo-de-goma-grit-n-juguete-con-chirrido-juguete-divertido-con-sonido-Al-apretar-para.jpg" width="400" height="790">


 

   
# Outline

- Decision Trees 
- Gradient Boosting
- XGBoost
    - Qué es XGBoost?
    - Características principales
    - Por qué usar XGBoost?
    - Qué algoritmo se usa?
    - Instalación (aunque este stand-up no es un tutorial)
- Nuestro primero modelo 🧶🐈
    - Descripción del problema
    - Carga de datos
    - Entrenamiento del modelo
    - Generación de predicciones ($\hat{Y}$)
    
- Visualización de árboles de decisión e imporancia de features
    




- Feature Importance
    - Plot
    - Tree Graph
    -

# Decision Trees

# Gradient Boosting

# XGBoost


## XGBoost Features



x

## System Features

- Paralelización
- Cómputo Distribuido
- Out-of-Core Computing
- Cache: Optimización de est

## Algorithm Features

-

## Por qué usar XGBoost?


In [1]:
fruit = ['apple', 'banana', 'orange', 'lime']